In [1]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import OPTICS
from sklearn.neighbors import NearestNeighbors
from torch.utils.data import DataLoader, TensorDataset
from sentence_transformers import SentenceTransformer
import warnings
import random

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# ─── CONFIG ────────────────────────────────────────────────────────────────────
CONFIG = {
    'data_path':               r"C:\Users\PC\Downloads\merged_movies (3).csv",
    'embeddings_path':         "new_embeddings_distilroberta.npy",
    'output_dir':              "movies_clusters_optics",
    'umap_components':         15,
    'umap_n_neighbors':        75,
    'umap_min_dist':           0.1,
    'umap_metric':             'cosine',
    'latent_dim':              64,
    'autoencoder_epochs':      50,
    'denoising_epochs':        15,
    'autoencoder_batch_size':  256,
    'denoising_noise_std':     0.05,
    'optics_min_samples':      50,  # Minimum points for core point, matches HDBSCAN/DBSCAN
    'optics_xi':               0.05,  # Steepness for hierarchical clustering
    'optics_metric':           'euclidean',
    'noise_reassignment':      True,  # Reassign noise points to nearest cluster
    'reassignment_percentile': 90,  # Distance percentile for noise reassignment
    'genre_weight':            1.5,
    'runtime_weight':          0.1,
    'year_weight':             0.3,
    'sbert_weight':            1.0,
    'silhouette_sample_size':   10000
}

# Create output directory
os.makedirs(CONFIG['output_dir'], exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── 1. LOAD & FILTER DATA ─────────────────────────────────────────────────────
def load_and_filter_data(config):
    """Load and preprocess movie dataset."""
    df = pd.read_csv(config['data_path']).drop_duplicates('MovieID').reset_index(drop=True)
    if 'Genre:(no genres listed)' in df.columns:
        df = df[df['Genre:(no genres listed)'] == False].reset_index(drop=True)
    df['release_year'] = pd.to_numeric(df.get('release_year', np.nan), errors='coerce')
    df['release_year'] = df['release_year'].fillna(df['release_year'].median()).clip(1900, 2025)
    df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce').fillna(df['runtime'].median())
    
    # Log genre distribution
    genre_cols = [c for c in df.columns if c.startswith('Genre:')]
    print(f"Total genre columns in dataset: {len(genre_cols)}")
    genre_counts = df[genre_cols].sum().sort_values(ascending=False)
    print("Top 5 genres in dataset:")
    for i in range(min(5, len(genre_counts))):
        print(f"{genre_counts.index[i]}: {int(genre_counts.iloc[i])} movies")
    single_genre_movies = df[genre_cols].sum(axis=1) == 1
    drama_only = (df['Genre:Drama'] == 1) & (df[genre_cols].sum(axis=1) == 1)
    print(f"Movies with only one genre: {single_genre_movies.sum()} ({single_genre_movies.sum()/len(df)*100:.2f}%)")
    print(f"Movies with only Drama genre: {drama_only.sum()} ({drama_only.sum()/len(df)*100:.2f}%)")
    
    return df, genre_cols

df, genre_cols = load_and_filter_data(CONFIG)

# ─── 2. LOAD / COMPUTE SBERT EMBEDDINGS ────────────────────────────────────────
def compute_sbert_embeddings(df, config):
    """Load or compute SBERT embeddings for movie overviews."""
    cache_path = os.path.join(config['output_dir'], 'sbert_embeddings.npy')
    if os.path.exists(cache_path):
        return np.load(cache_path)
    if os.path.exists(config['embeddings_path']):
        embeddings = np.load(config['embeddings_path'])
    else:
        st_model = SentenceTransformer('distilroberta-base', device=device)
        embeddings = st_model.encode(
            df['overview'].fillna(''),
            batch_size=config['autoencoder_batch_size'],
            show_progress_bar=True,
            convert_to_numpy=True
        )
        np.save(config['embeddings_path'], embeddings)
    np.save(cache_path, embeddings)
    return embeddings

sbert_embeddings = compute_sbert_embeddings(df, CONFIG)

# ─── 3. DENOISING AUTOENCODER ────────────────────────────────────────────────────
class DenoiseAE(nn.Module):
    """Denoising Autoencoder for SBERT embeddings."""
    def __init__(self, inp, lat):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(inp, 512), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(512, lat), nn.ReLU()
        )
        self.dec = nn.Sequential(
            nn.Linear(lat, 512), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(512, inp)
        )
    def forward(self, x):
        z = self.enc(x)
        return self.dec(z), z

def train_denoiser(emb, config):
    """Train denoising autoencoder and return latent representations."""
    cache_path = os.path.join(config['output_dir'], 'sbert_denoised.npy')
    if os.path.exists(cache_path):
        return np.load(cache_path)
    
    model = DenoiseAE(emb.shape[1], config['latent_dim']).to(device)
    opt = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
    loader = DataLoader(
        TensorDataset(torch.tensor(emb, dtype=torch.float32)),
        batch_size=config['autoencoder_batch_size'], shuffle=True
    )
    best_loss, patience, counter = float('inf'), 3, 0
    for epoch in range(config['denoising_epochs']):
        model.train()
        total = 0.0
        for (batch,) in loader:
            batch = batch.to(device)
            noisy = batch + torch.randn_like(batch) * config['denoising_noise_std']
            recon, _ = model(noisy)
            loss = nn.MSELoss()(recon, batch)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.item()
        avg = total / len(loader)
        if avg < best_loss:
            best_loss, counter = avg, 0
            torch.save(model.state_dict(), os.path.join(config['output_dir'], 'denoiser.pth'))
        else:
            counter += 1
            if counter >= patience:
                break
    model.load_state_dict(torch.load(os.path.join(config['output_dir'], 'denoiser.pth')))
    model.eval()
    with torch.no_grad():
        _, den = model(torch.tensor(emb, dtype=torch.float32).to(device))
    den = den.cpu().numpy()
    np.save(cache_path, den)
    return den

sbert_denoised = train_denoiser(sbert_embeddings, CONFIG)

# ─── 4. UMAP REDUCTION ─────────────────────────────────────────────────────────
def reduce_umap(embeddings, config):
    """Apply UMAP reduction to denoised embeddings."""
    cache_path = os.path.join(config['output_dir'], 'sbert_reduced.npy')
    if os.path.exists(cache_path):
        return np.load(cache_path)
    
    um = umap.UMAP(
        n_components=config['umap_components'],
        n_neighbors=config['umap_n_neighbors'],
        min_dist=config['umap_min_dist'],
        metric=config['umap_metric'],
        random_state=42
    )
    umap_emb = um.fit_transform(embeddings)
    reduced = MinMaxScaler().fit_transform(umap_emb)
    np.save(cache_path, reduced)
    
    # Visualize UMAP projection
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x=reduced[:10000, 0], y=reduced[:10000, 1], s=10, alpha=0.5)
    plt.title('UMAP Projection of Denoised SBERT Embeddings (First 10,000 Movies)')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.savefig(os.path.join(config['output_dir'], 'umap_projection.png'), dpi=300)
    plt.close()
    
    return reduced

sbert_reduced = reduce_umap(sbert_denoised, CONFIG)

# ─── 5. HYBRID FEATURE ENGINEERING ─────────────────────────────────────────────
def make_hybrid(df, sbert_red, genre_cols, config):
    """Create hybrid features from genres, runtime, year, and SBERT embeddings."""
    # Compute genre weights
    w = np.log(len(df) / (df[genre_cols].sum() + 1))
    w = MinMaxScaler().fit_transform(w.values.reshape(-1, 1)).flatten()
    w[genre_cols.index('Genre:Drama')] *= 0.7
    
    # Log genre variances
    gf = df[genre_cols].values * w * config['genre_weight']
    genre_variances = np.var(gf, axis=0)
    variance_df = pd.DataFrame({
        'Genre': genre_cols,
        'Variance': genre_variances
    })
    print("\nGenre feature variances:")
    print(variance_df.sort_values('Variance').to_string(index=False))
    variance_df.to_csv(os.path.join(config['output_dir'], 'genre_variances.csv'), index=False)
    
    # Scale other features
    y = df['release_year'].values.reshape(-1, 1)
    yf = MinMaxScaler().fit_transform(y) * config['year_weight']
    r = df['runtime'].values.reshape(-1, 1)
    rf = MinMaxScaler().fit_transform(r) * config['runtime_weight']
    
    hybrid = np.hstack([gf, yf, rf, sbert_red * config['sbert_weight']])
    hybrid = MinMaxScaler().fit_transform(hybrid)
    
    # Validate feature variance
    feature_variances = np.var(hybrid, axis=0)
    print(f"Feature variances (min: {feature_variances.min():.4f}, max: {feature_variances.max():.4f}, mean: {feature_variances.mean():.4f})")
    if feature_variances.min() < 1e-4:
        print("Warning: Some features have near-zero variance, consider removing them.")
    
    return hybrid, genre_cols

hybrid, valid_genre_cols = make_hybrid(df, sbert_reduced, genre_cols, CONFIG)

# ─── 6. COMPRESS HYBRID FEATURES ───────────────────────────────────────────────
class HybridAE(nn.Module):
    """Autoencoder for compressing hybrid features."""
    def __init__(self, inp, lat):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(inp, 256), nn.ReLU(),
            nn.Linear(256, lat), nn.ReLU()
        )
        self.dec = nn.Sequential(
            nn.Linear(lat, 256), nn.ReLU(),
            nn.Linear(256, inp)
        )
    def forward(self, x):
        z = self.enc(x)
        return self.dec(z), z

def train_hybrid(ftrs, config):
    """Train hybrid autoencoder and return latent representations."""
    cache_path = os.path.join(config['output_dir'], 'latent_features.npy')
    if os.path.exists(cache_path):
        return np.load(cache_path)
    
    model = HybridAE(ftrs.shape[1], config['latent_dim']).to(device)
    opt = optim.Adam(model.parameters(), lr=5e-4)
    loader = DataLoader(
        TensorDataset(torch.tensor(ftrs, dtype=torch.float32)),
        batch_size=config['autoencoder_batch_size'], shuffle=True
    )
    best, patience, counter = float('inf'), 5, 0
    for epoch in range(config['autoencoder_epochs']):
        model.train()
        total = 0.0
        for (batch,) in loader:
            batch = batch.to(device)
            recon, _ = model(batch)
            loss = nn.MSELoss()(recon, batch)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.item()
        avg = total / len(loader)
        if avg < best:
            best, counter = avg, 0
            torch.save(model.state_dict(), os.path.join(config['output_dir'], 'hybrid_ae.pth'))
        else:
            counter += 1
            if counter >= patience:
                break
    model.load_state_dict(torch.load(os.path.join(config['output_dir'], 'hybrid_ae.pth')))
    model.eval()
    with torch.no_grad():
        _, latent = model(torch.tensor(ftrs, dtype=torch.float32).to(device))
    latent = latent.cpu().numpy()
    np.save(cache_path, latent)
    return latent

latent = train_hybrid(hybrid, CONFIG)

# ─── 7. OPTICS CLUSTERING ──────────────────────────────────────────────────────
def cluster_optics(latent, config):
    """Apply OPTICS clustering with optional noise reassignment."""
    print("Running OPTICS clustering...")
    optics = OPTICS(
        min_samples=config['optics_min_samples'],
        xi=config['optics_xi'],
        metric=config['optics_metric'],
        n_jobs=-1
    )
    labels = optics.fit_predict(latent)
    
    # Log initial clustering results
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = np.sum(labels == -1)
    print(f"Initial OPTICS clustering completed: {n_clusters} clusters, {n_noise} noise points ({n_noise/len(labels)*100:.2f}%)")
    
    # Reassign noise points to nearest cluster
    if config['noise_reassignment'] and n_noise > 0:
        print("Reassigning noise points...")
        # Compute distances to core points
        core_mask = labels != -1
        core_points = latent[core_mask]
        core_labels = labels[core_mask]
        noise_mask = labels == -1
        noise_points = latent[noise_mask]
        
        # Find nearest core point for each noise point
        nn = NearestNeighbors(n_neighbors=1, metric=config['optics_metric'], n_jobs=-1)
        nn.fit(core_points)
        distances, indices = nn.kneighbors(noise_points)
        
        # Use 90th percentile distance as threshold
        distance_threshold = np.percentile(distances, config['reassignment_percentile'])
        new_labels = labels.copy()
        for i, (dist, idx) in enumerate(zip(distances.flatten(), indices.flatten())):
            if dist <= distance_threshold:
                new_labels[noise_mask][i] = core_labels[idx]
        
        # Update noise count
        n_noise_new = np.sum(new_labels == -1)
        print(f"Noise reassignment completed: {n_noise_new} noise points remain ({n_noise_new/len(labels)*100:.2f}%)")
        labels = new_labels
    
    return labels

labels = cluster_optics(latent, CONFIG)

# ─── 8. CLUSTER SIZE REPORTING ─────────────────────────────────────────────────
def report_clusters(df, labels, genre_cols, config):
    """Generate and save cluster statistics and visualizations."""
    cluster_sizes = pd.Series(labels[labels != -1]).value_counts().sort_values(ascending=False)
    print("\nTop 5 largest clusters (excluding noise):")
    for i in range(5):
        if i < len(cluster_sizes):
            print(f"Cluster {cluster_sizes.index[i]}: {cluster_sizes.iloc[i]} movies")
    
    # Save largest cluster (if exists)
    if len(cluster_sizes) > 0:
        largest_cluster_id = cluster_sizes.index[0]
        largest_cluster_movies = df[labels == largest_cluster_id][['MovieID', 'title', 'release_year', 'runtime'] + genre_cols]
        largest_cluster_movies.to_csv(os.path.join(config['output_dir'], f'cluster_{largest_cluster_id}_movies.csv'), index=False)
        print(f"Saved largest cluster (Cluster {largest_cluster_id}) movies to: {config['output_dir']}/cluster_{largest_cluster_id}_movies.csv")
        genre_counts = largest_cluster_movies[genre_cols].sum().sort_values(ascending=False)
        print(f"\nTotal genre columns in Cluster {largest_cluster_id}: {len(genre_cols)}")
        print(f"Non-zero genre columns in Cluster {largest_cluster_id}: {sum(genre_counts > 0)}")
        print(f"\nTop 5 genres in Cluster {largest_cluster_id}:")
        for i in range(min(5, len(genre_counts))):
            print(f"{genre_counts.index[i]}: {int(genre_counts.iloc[i])} movies")
    
    return cluster_sizes

cluster_sizes = report_clusters(df, labels, genre_cols, CONFIG)

# ─── 9. FINAL METRICS & VISUALIZATION ──────────────────────────────────────────
def compute_metrics_and_visualize(df, latent, labels, cluster_sizes, config):
    """Compute clustering metrics and generate visualizations."""
    # Compute metrics (only for clustered points)
    clustered_mask = labels != -1
    clustered_latent = latent[clustered_mask]
    clustered_labels = labels[clustered_mask]
    np.random.seed(42)
    idx = np.random.choice(len(clustered_latent), min(len(clustered_latent), config['silhouette_sample_size']), replace=False)
    sil_score, db_index, ch_score = 0.0, np.inf, 0.0
    if len(np.unique(clustered_labels[idx])) > 1:
        sil_score = silhouette_score(clustered_latent[idx], clustered_labels[idx])
        db_index = davies_bouldin_score(clustered_latent[idx], clustered_labels[idx])
        ch_score = calinski_harabasz_score(clustered_latent[idx], clustered_labels[idx])
    
    # Cluster statistics
    num_clusters = len(cluster_sizes)
    total_movies = len(labels)
    movies_in_clusters = np.sum(clustered_mask)
    percentage_in_clusters = (movies_in_clusters / total_movies) * 100
    n_noise = total_movies - movies_in_clusters
    avg_size = cluster_sizes.mean() if num_clusters > 0 else 0
    min_size = cluster_sizes.min() if num_clusters > 0 else 0
    max_size = cluster_sizes.max() if num_clusters > 0 else 0
    median_size = cluster_sizes.median() if num_clusters > 0 else 0
    
    # Generate report
    report_lines = []
    report_lines.append("========== Movie Clustering Analysis Report (OPTICS) ==========\n")
    report_lines.append("--- Data Summary ---\n")
    report_lines.append(f"Total number of movies: {total_movies}\n")
    report_lines.append(f"Clustering performed on: {config['latent_dim']}-dimensional latent space from hybrid features\n")
    report_lines.append("--- Clustering Summary ---\n")
    report_lines.append(f"Number of clusters: {num_clusters}\n")
    report_lines.append(f"Movies assigned to clusters: {movies_in_clusters} ({percentage_in_clusters:.2f}%)\n")
    report_lines.append(f"Noise points: {n_noise} ({n_noise/total_movies*100:.2f}%)\n")
    report_lines.append(f"Average cluster size: {avg_size:.1f}\n")
    report_lines.append(f"Minimum cluster size: {min_size}\n")
    report_lines.append(f"Maximum cluster size: {max_size}\n")
    report_lines.append(f"Median cluster size: {median_size}\n")
    report_lines.append("--- Top 5 Largest Clusters ---\n")
    for i, (cluster, size) in enumerate(cluster_sizes.items()):
        if i < 5:
            report_lines.append(f"Cluster {cluster}: {size} movies\n")
    report_lines.append("\n--- Top 5 Smallest Clusters ---\n")
    for i, (cluster, size) in enumerate(cluster_sizes.sort_values().items()):
        if i < 5:
            report_lines.append(f"Cluster {cluster}: {size} movies\n")
    
    # Sample movies
    random.seed(42)
    sample_clusters = random.sample(list(cluster_sizes.index), min(3, num_clusters)) if num_clusters > 0 else []
    report_lines.append("\n--- Sample Movies from 3 Random Clusters ---\n")
    for cluster in sample_clusters:
        cluster_movies = df[labels == cluster]['title'].tolist()
        sample_titles = random.sample(cluster_movies, min(7, len(cluster_movies)))
        report_lines.append(f"\nCluster {cluster} (size: {cluster_sizes[cluster]}):\n")
        for title in sample_titles:
            report_lines.append(f" - {title}\n")
    
    # Quality metrics
    report_lines.append("\n--- Clustering Quality Metrics (Clustered Points Only) ---\n")
    report_lines.append(f"{'Silhouette Score (subsample)':<30}: {sil_score:.4f}\n")
    report_lines.append(f"{'Davies–Bouldin Index':<30}: {db_index:.4f}\n")
    report_lines.append(f"{'Calinski–Harabasz Score':<30}: {ch_score:.2f}\n")
    report_lines.append("==================================================\n")
    
    # Print and save report
    with open(os.path.join(config['output_dir'], 'clustering_report.txt'), 'w') as f:
        for line in report_lines:
            print(line, end='')
            f.write(line)
    
    # Save cluster assignments
    pd.DataFrame({'MovieID': df['MovieID'], 'cluster': labels}).to_csv(
        os.path.join(config['output_dir'], 'movie_clusters_optics.csv'), index=False
    )
    
    # Save JSON summary
    summary = {
        'total_movies': int(total_movies),
        'num_clusters': int(num_clusters),
        'movies_in_clusters': int(movies_in_clusters),
        'percentage_in_clusters': float(percentage_in_clusters),
        'noise_points': int(n_noise),
        'percentage_noise': float(n_noise/total_movies*100),
        'avg_cluster_size': float(avg_size),
        'min_cluster_size': int(min_size),
        'max_cluster_size': int(max_size),
        'median_cluster_size': float(median_size),
        'silhouette_score': float(sil_score),
        'davies_bouldin_index': float(db_index),
        'calinski_harabasz_score': float(ch_score)
    }
    with open(os.path.join(config['output_dir'], 'clustering_summary.json'), 'w') as f:
        json.dump(summary, f, indent=4)
    
    # Visualize clusters in UMAP space
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    umap_2d = umap_reducer.fit_transform(latent)
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(
        umap_2d[:, 0], umap_2d[:, 1],
        c=labels, cmap='Spectral', s=5, alpha=0.6
    )
    plt.colorbar(scatter, label='Cluster ID (-1 = Noise)')
    plt.title('UMAP Visualization of Movie Clusters (OPTICS)')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.savefig(os.path.join(config['output_dir'], 'cluster_visualization.png'), dpi=300)
    plt.close()
    
    # Generate cluster genre distribution table
    cluster_genre_dist = []
    for cluster in cluster_sizes.index:
        cluster_movies = df[labels == cluster]
        genre_counts = cluster_movies[genre_cols].sum().to_dict()
        genre_counts['Cluster'] = int(cluster)
        genre_counts['Size'] = int(cluster_sizes[cluster])
        cluster_genre_dist.append(genre_counts)
    pd.DataFrame(cluster_genre_dist).to_csv(
        os.path.join(config['output_dir'], 'cluster_genre_distribution.csv'), index=False
    )
    
    print(f"\nSaved clustering report to: {config['output_dir']}/clustering_report.txt")
    print(f"Saved cluster assignments to: {config['output_dir']}/movie_clusters_optics.csv")
    print(f"Saved clustering summary to: {config['output_dir']}/clustering_summary.json")
    print(f"Saved cluster visualization to: {config['output_dir']}/cluster_visualization.png")
    print(f"Saved genre distribution to: {config['output_dir']}/cluster_genre_distribution.csv")
    print(f"Saved genre variances to: {config['output_dir']}/genre_variances.csv")

compute_metrics_and_visualize(df, latent, labels, cluster_sizes, CONFIG)

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total genre columns in dataset: 20
Top 5 genres in dataset:
Genre:Drama: 19750 movies
Genre:Comedy: 12380 movies
Genre:Thriller: 8275 movies
Genre:Action: 6631 movies
Genre:Horror: 6335 movies
Movies with only one genre: 18774 (41.41%)
Movies with only Drama genre: 6667 (14.71%)

Genre feature variances:
                   Genre  Variance
             Genre:Drama  0.000000
Genre:(no genres listed)  0.000000
            Genre:Comedy  0.000996
              Genre:IMAX  0.002022
         Genre:Film-Noir  0.002445
          Genre:Thriller  0.002597
            Genre:Action  0.003420
            Genre:Horror  0.003574
           Genre:Romance  0.003721
           Genre:Musical  0.004012
           Genre:Western  0.004267
       Genre:Documentary  0.004543
             Genre:Crime  0.004575
               Genre:War  0.004754
         Genre:Adventure  0.004912
            Genre:Sci-Fi  0.004920
         Genre:Animation  0.005101
          Genre:Children  0.005113
           Genre:Mystery  0.0

MemoryError: Unable to allocate 10.1 MiB for an array with shape (20609, 64) and data type float64